# Planoconvex lens (piecewise line)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        self.shape = tlm.PiecewiseLine(40., X=nn.Parameter(torch.linspace(0.0, 4.0, 20)))

        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.5), outer_thickness=0.5)

        self.optics = tlm.OpticalSequence(
            tlm.PointSourceAtInfinity(beam_diameter=40),
            tlm.Gap(10.),
            
            self.lens,
            
            tlm.Gap(45.0),
            tlm.FocalPoint(),
        )

    def forward(self, inputs, sampling):
        return self.optics(inputs, sampling)

optics = Optics()

tlm.render_plt(optics)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-2),
    sampling = {"rays": 50},
    num_iter = 100
)

tlm.render_plt(optics, sampling={"rays": 50})